<a href="https://colab.research.google.com/github/sanjaynagi/AnoFold/blob/main/notebooks/AnoFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %pip install py3Dmol

# !wget https://github.com/gnina/gnina/releases/download/v1.1/gnina
# !chmod +x gnina
# !./gnina --version

In [2]:
# # genes = ['AGAP006227', 'AGAP006228', 'AGAP006723', 'AGAP006724', 'AGAP006725', 'AGAP006726', 'AGAP006727']
# df_genes = pd.read_csv("https://raw.githubusercontent.com/sanjaynagi/AnoExpress/main/resources/AgamP4.annots.tsv", 
#                        sep="\t")

# search = 'S-transferase'
# genes = df_genes.fillna('bl').query('GeneDescription.str.contains(@search)')['GeneID'].to_list()

In [3]:
# ligands = ['ddt3d', 'deltamethrin3d']           #, 'pirimiphos-methyl-oxon3d', 'malathion3d', 'cis-permethrin3d','transfluthrin3d']
# genes = ['AGAP006227']
# # for gene in genes:
# #     for ligand in ligands:
# #           res = invin(gene, ligand=ligand, override_desc='glutathione transferase')

## iMOVE - inverse molecular docking with alphafold and gnina

In [2]:
import imove

mutagen_dicts = [None, {114:'T'}, {119:'V'}, {120:'L'}]

In [3]:
# for ligand in ['ddt3d', 'deltamethrin3d']:
#     for mutagen_dict in mutagen_dicts:
        
#         # if I114T, the motif needs to be adjusted to include the new mutation
#         override_motif = None
#         if mutagen_dict:
#             if list(mutagen_dict.keys())[0] == 114:
#                 override_motif = ('RFTFERI', "O", 4)

#         imove.dock("AGAP009194", ligand, wkdir="../../", verbose=True, mutagenesis_dict=mutagen_dict, override_motif=override_motif)

In [8]:
ls ../..

docked/         figures/  ligands/  pyproject.toml  receptors/
example_df.csv  imove/    logs/     README.md       resources/


In [1]:
import imove
for mutagenesis_dict in [None, {224:'I'}]:
    docked = imove.dock("AGAP000818",
                        "deltamethrin3d", 
                        wkdir="../../", 
                        verbose=True, 
                        mutagenesis_dict=mutagenesis_dict,
                        override_motif=None,
                        p450=True
                       )

Starting docking process for gene_id: AGAP000818, ligand: deltamethrin3d
receptor file found...
Active site coordinates: x=-15.846, y=-23.032, z=-11.293
Skipping GNINA docking: log file already exists or ligand file is missing


UnboundLocalError: cannot access local variable 'active_site_motif' where it is not associated with a value

In [15]:
from rdkit.Chem.PandasTools import LoadSDF
df = LoadSDF(sdf_file, smilesName='SMILES')

In [20]:
df['SMILES'][0]

'CC1(C)[C@H](C(=O)O[C@H](C#N)c2cccc(Oc3ccccc3)c2)[C@@H]1C=C(Br)Br'

In [26]:
from rdkit import Chem
from rdkit.Chem import AllChem
from typing import List, Tuple

def get_atoms_midpoint(conf, atom_indices: List[int]) -> Tuple[float, float, float]:
    """
    Calculate the 3D midpoint of any number of atoms.
    
    :param conf: RDKit conformer object
    :param atom_indices: List of atom indices
    :return: Tuple representing the 3D coordinates of the midpoint
    """
    if not atom_indices:
        raise ValueError("No atom indices provided")
    
    total_x, total_y, total_z = 0, 0, 0
    for idx in atom_indices:
        pos = conf.GetAtomPosition(idx)
        total_x += pos.x
        total_y += pos.y
        total_z += pos.z
    
    num_atoms = len(atom_indices)
    midpoint = (total_x / num_atoms, total_y / num_atoms, total_z / num_atoms)
    return midpoint

def measure_distance_to_atoms(mol, residue_coords, smarts_pattern, conf_id=-1):
    """
    Measure the distance from a residue to atoms specified by SMARTS.
    
    :param mol: RDKit molecule object
    :param residue_coords: Tuple of (x, y, z) coordinates of the residue
    :param smarts_pattern: SMARTS pattern specifying the atoms of interest
    :param conf_id: Conformer ID to use (-1 for default conformer)
    :return: Distance from the residue to the midpoint of matching atoms
    """
    matches = mol.GetSubstructMatches(Chem.MolFromSmarts(smarts_pattern))
    if not matches:
        print(f"Warning: No matches found for pattern {smarts_pattern}")
        return None
    
    conf = mol.GetConformer(conf_id)
    distances = []
    
    for match in matches:
        midpoint = get_atoms_midpoint(conf, match)
        distance = sum((a - b) ** 2 for a, b in zip(residue_coords, midpoint)) ** 0.5
        distances.append(distance)
    
    return min(distances) if distances else None

def process_sdf_conformations(sdf_file, residue_coords, smarts_pattern):
    """
    Process all conformations in an SDF file and measure distances to specified atoms.
    
    :param sdf_file: Path to the SDF file
    :param residue_coords: Tuple of (x, y, z) coordinates of the residue
    :param smarts_pattern: SMARTS pattern specifying the atoms of interest
    :return: List of distances for each conformation
    """
    suppl = Chem.SDMolSupplier(sdf_file, removeHs=False)
    all_distances = []

    for mol in suppl:
        if mol is not None:
            for conf_id in range(mol.GetNumConformers()):
                distance = measure_distance_to_atoms(mol, residue_coords, smarts_pattern, conf_id)
                all_distances.append(distance)
    
    return all_distances

# Example usage and testing
def test_midpoint_calculation(smiles, smarts_pattern):
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)  # Generate 3D coordinates
    conf = mol.GetConformer()
    
    matches = mol.GetSubstructMatches(Chem.MolFromSmarts(smarts_pattern))
    print(f"SMILES: {smiles}")
    print(f"SMARTS: {smarts_pattern}")
    
    for match in matches:
        midpoint = get_atoms_midpoint(conf, match)
        print(f"Atoms: {match}")
        print(f"Midpoint: {midpoint}")
    print()

# Test cases
test_midpoint_calculation("CC(=O)OC1=CC=CC=C1", "O=C-O")  # Ester group
test_midpoint_calculation("CC1=CC=CC=C1", "c1ccccc1")     # Benzene ring
test_midpoint_calculation("CC(C)C", "C(C)C")              # Isopropyl group

# Example of using the updated functions with an SDF file
sdf_file = "../../docked/AGAP006227_deltamethrin3d.sdf"
residue_coords = (1.0, 2.0, 3.0)  # Example residue coordinates

ester_pattern = "C(=O)O"
benzene_pattern = "c1ccccc1"

ester_distances = process_sdf_conformations(sdf_file, residue_coords, ester_pattern)
benzene_distances = process_sdf_conformations(sdf_file, residue_coords, benzene_pattern)

print("Distances to ester group for each conformation:")
for i, distance in enumerate(ester_distances):
    print(f"Conformation {i+1}: {distance:.2f}")

print("\nDistances to benzene ring for each conformation:")
for i, distance in enumerate(benzene_distances):
    print(f"Conformation {i+1}: {distance:.2f}")



SMILES: CC(=O)OC1=CC=CC=C1
SMARTS: O=C-O
Atoms: (2, 1, 3)
Midpoint: (1.5877753595093242, 0.4822550828808246, -0.07054475168912559)

SMILES: CC1=CC=CC=C1
SMARTS: c1ccccc1
Atoms: (1, 2, 3, 4, 5, 6)
Midpoint: (-0.39946509999666624, -0.005751985466239969, -0.04981439393874695)

SMILES: CC(C)C
SMARTS: C(C)C
Atoms: (1, 0, 2)
Midpoint: (0.06883793183177578, -0.4763101155974705, 0.040550998455633654)
Atoms: (1, 0, 3)
Midpoint: (0.3751832287009016, 0.29429690260557567, 0.03908521937897735)
Atoms: (1, 2, 3)
Midpoint: (-0.441097591435252, 0.18495223975211175, 0.040316832386537646)

Distances to ester group for each conformation:
Conformation 1: 15.09
Conformation 2: 13.55
Conformation 3: 10.91
Conformation 4: 11.20
Conformation 5: 10.14
Conformation 6: 9.45
Conformation 7: 10.90
Conformation 8: 15.28
Conformation 9: 10.30

Distances to benzene ring for each conformation:
Conformation 1: 7.93
Conformation 2: 8.92
Conformation 3: 8.76
Conformation 4: 8.71
Conformation 5: 10.65
Conformation 6: 8.98


[10:50:30] Molecule does not have explicit Hs. Consider calling AddHs()
[10:50:30] Molecule does not have explicit Hs. Consider calling AddHs()
[10:50:30] Molecule does not have explicit Hs. Consider calling AddHs()
